In [1]:
import seaborn as sns
import pandas as pd
import numpy as np
import altair as alt
from markdown import markdown
from IPython.display import Markdown
from ipywidgets.widgets import HTML, Tab
from ipywidgets import widgets
from datetime import timedelta
from matplotlib import pyplot as plt
import os.path as op

from mod import load_data, alt_theme

from warnings import simplefilter
simplefilter('ignore')

In [2]:
def author_url(author):
    return f"https://github.com/{author}"

In [3]:
# Parameters
fmt_date = "{:%Y-%m-%d}"

n_days = 90
start_date = fmt_date.format(pd.datetime.today() - timedelta(days=n_days))
end_date = fmt_date.format(pd.datetime.today())

renderer = "jupyterlab"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples", "nteract"]

In [4]:
# Parameters
renderer = "kaggle"
github_orgs = ["jupyterhub", "jupyter", "jupyterlab", "jupyter-widgets", "ipython", "binder-examples"]
start_date = "2019-07-01"
end_date = "2019-08-01"


In [5]:
comments, issues, prs = load_data('../data/')
bot_names = pd.read_csv('bot_names.csv')['names'].tolist()
comments = comments.query('author not in @bot_names').drop_duplicates()
issues = issues.query('author not in @bot_names').drop_duplicates()
prs = prs.query('author not in @bot_names').drop_duplicates()

In [6]:
# Only keep the datapoints we want
query_data = 'updatedAt > @start_date and updatedAt < @end_date and org in @github_orgs'
comments = comments.query(query_data)
issues = issues.query(query_data)
prs = prs.query(query_data)

In [7]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [8]:
# Information about out time window
time_delta = pd.to_datetime(end_date) - pd.to_datetime(start_date)
n_days = time_delta.days

# Information about the data we loaded
github_orgs = comments['org'].unique()

# GitHub activity

Jupyter also has lots of activity across GitHub repositories. The following sections contain
overviews of recent activity across the following GitHub organizations:

In [9]:
# Define colors we'll use for GitHub membership
author_types = ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', "NONE"]

author_palette = sns.palettes.blend_palette(["lightgrey", "lightgreen", "darkgreen"], 4)
author_colors = ["rgb({}, {}, {}, {})".format(*(ii*256)) for ii in author_palette]
author_color_dict = {key: val for key, val in zip(author_types, author_palette)}

In [10]:
orgs_md = []
for org in github_orgs:
    orgs_md.append(f'* [github.com/{org}](https://github.com/{org})')
Markdown('\n'.join(orgs_md))

* [github.com/jupyterlab](https://github.com/jupyterlab)
* [github.com/ipython](https://github.com/ipython)
* [github.com/jupyter-widgets](https://github.com/jupyter-widgets)
* [github.com/binder-examples](https://github.com/binder-examples)
* [github.com/jupyter](https://github.com/jupyter)
* [github.com/jupyterhub](https://github.com/jupyterhub)

In [11]:
Markdown(f"Showing GitHub activity from **{start_date}** to **{end_date}**")

Showing GitHub activity from **2019-07-01** to **2019-08-01**

## List of all contributors per organization

First, we'll list each contributor that has contributed to each organization in the last several days.
Contributions to open source projects are diverse, and involve much more than just contributing code and
code review. Thanks to everybody in the Jupyter communities for all that they do.

In [12]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, org in enumerate(github_orgs):
    authors_comments = comments.query('org == @org')['author']
    authors_prs = prs.query('org == @org')['author']
    unique_participants = np.unique(np.hstack([authors_comments.values, authors_prs.values]).astype(str)).tolist()
    unique_participants.sort(key=lambda a: a.lower())
    
    all_participants = [f"[{participant}](https://github.com/{participant})" for participant in unique_participants]
    participants_md = " | ".join(all_participants)
    md_html = HTML("<center>{}</center>".format(markdown(participants_md)))
    
    children = list(tabs.children)
    children.append(md_html)
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"All participants across issues and pull requests in each org in the last {n_days} days"))
display(tabs)

All participants across issues and pull requests in each org in the last 31 days

## Merged Pull requests

Here's an analysis of **merged pull requests** across each of the repositories in the Jupyter
ecosystem.

In [13]:
merged = prs.query('state == "MERGED" and closedAt > @start_date and closedAt < @end_date')

In [14]:
prs_by_repo = merged.groupby(['org', 'repo']).count()['author'].reset_index().sort_values(['org', 'author'], ascending=False)
alt.Chart(data=prs_by_repo, title=f"Merged PRs in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=prs_by_repo['repo'].values.tolist()),
    y='author',
    color='org'
)

Chart({
  data:                 org                      repo  author
  55       jupyterlab                jupyterlab      51
  57       jupyterlab  jupyterlab-data-explorer       9
  54       jupyterlab         jupyter-renderers       5
  58       jupyterlab           jupyterlab-demo       5
  59       jupyterlab            jupyterlab-git       5
  ..              ...                       ...     ...
  3           ipython                   ipython      10
  1           ipython                 ipykernel       3
  2           ipython               ipyparallel       2
  4           ipython                 traitlets       2
  0   binder-examples                data-quilt       1
  
  [64 rows x 3 columns],
  encoding: FacetedEncoding({
    color: Color({
      field: 'org',
      type: 'nominal'
    }),
    x: X({
      field: 'repo',
      sort: ['jupyterlab', 'jupyterlab-data-explorer', 'jupyter-renderers', 'jupyterlab-demo', 'jupyterlab-git', 'debugger', 'jupyterlab-github', 'extension-cookiecutter-ts', 'jupyterlab-commenting', 'theme-cookiecutter', 'scipy2019-jupyterlab-tutorial', 'extension-cookiecutter-js', 'mimerender-cookiecutter-ts', 'mybinder.org-deploy', 'jupyterhub', 'binderhub', 'the-littlest-jupyterhub', 'batchspawner', 'team-compass', 'kerberosauthenticator', 'traefik-proxy', 'binder', 'jupyter-server-proxy', 'ldapauthenticator', 'yarnspawner', 'zero-to-jupyterhub-k8s', 'dockerspawner', 'firstuseauthenticator', 'jupyterlab-hub', 'kubespawner', 'ltiauthenticator', 'nbgitpuller', 'tutorial', 'ipywidgets', 'ipyleaflet', 'jupyterlab-sidecar', 'midicontrols', 'pythreejs', 'notebook', 'jupyter-book', 'telemetry', 'docker-stacks', 'repo2docker', 'enterprise_gateway', 'jupyter.github.io', 'jupyter_client', 'nbconvert', 'jupyter-sphinx', 'jupyter_server', 'qtconsole', 'jupyter', 'jupyter-packaging', 'design', 'nb2kg', 'nbviewer', 'governance', 'nbdime', 'ops-handbook', 'roadmap', 'ipython', 'ipykernel', 'ipyparallel', 'traitlets', 'data-quilt'],
      type: 'nominal'
    }),
    y: Y({
      field: 'author',
      type: 'quantitative'
    })
  }),
  mark: 'bar',
  title: 'Merged PRs in the last 31 days'
})

### A list of merged PRs by project

Below is a tabbed readout of recently-merged PRs. Check out the title to get an idea for what they
implemented, and be sure to thank the PR author for their hard work!

In [15]:
tabs = widgets.Tab(children=[])
merged_by = {}
pr_by = {}
for ii, (org, idata) in enumerate(merged.groupby('org')):
    issue_md = []
    issue_md.append(f"#### Closed PRs for org: `{org}`")
    issue_md.append("")
    for (org, repo), prs in idata.groupby(['org', 'repo']):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")
        for _, pr in prs.iterrows():
            user_name = pr['author']
            user_url = author_url(user_name)
            pr_number = pr['number']
            pr_html = pr['url']
            pr_title = pr['title']
            pr_closedby = pr['mergedBy']
            pr_closedby_url = f"https://github.com/{pr_closedby}"
            if user_name not in pr_by:
                pr_by[user_name] = 1
            else:
                pr_by[user_name] += 1
                
            if pr_closedby not in merged_by:
                merged_by[pr_closedby] = 1
            else:
                merged_by[pr_closedby] += 1
            text = f"* [(#{pr_number})]({pr_html}): _{pr_title}_ by **[@{user_name}]({user_url})** merged by **[@{pr_closedby}]({pr_closedby_url})**"
            issue_md.append(text)
    issue_md.append('')
    markdown_html = markdown('\n'.join(issue_md))
    
    children = list(tabs.children)
    children.append(HTML(markdown_html))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
tabs

### Authoring and merging stats by repository

Let's see who has been doing most of the PR authoring and merging. The PR author is generally the
person that implemented a change in the repository (code, documentation, etc). The PR merger is
the person that "pressed the green button" and got the change into the main codebase.

In [16]:
# Prep our merging DF
merged_by_repo = merged.groupby(['org', 'repo', 'author'], as_index=False).agg({'id': 'count', 'authorAssociation': 'first'}).rename(columns={'id': "authored", 'author': 'username'})
closed_by_repo = merged.groupby(['org', 'repo', 'mergedBy']).count()['id'].reset_index().rename(columns={'id': "closed", "mergedBy": "username"})

In [17]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(merged_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"PR authors for {iorg} in the last {n_days} days"
    idata = idata.groupby('username', as_index=False).agg({'authored': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('authored', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='authored',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)

alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:       username  authored authorAssociation
    0  matheusmota         1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:                  username  authored authorAssociation
    0                 Carreau         3            MEMBER
    1                   Jamim         3       CONTRIBUTOR
    2   LittleLightLittleFire         1       CONTRIBUTOR
    3                   MSeal         1       CONTRIBUTOR
    4              OriolAbril         1       CONTRIBUTOR
    5          brandonwillard         1       CONTRIBUTOR
    6                  ellert         1       CONTRIBUTOR
    7                 hroncok         1       CONTRIBUTOR
    8              juanis2112         1       CONTRIBUTOR
    9                 minho42         1       CONTRIBUTOR
    10                  minrk         1            MEMBER
    11                sunpoet         1       CONTRIBUTOR
    12             sysadmin75         1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'authored',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'PR authors for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:           username  authored authorAssociation
    12       choldgraf        13            MEMBER
    60       yuvipanda         7            MEMBER
    47           minrk         6            MEMBER
    4          Zsailer         5            MEMBER
    3    SylvainCorlay         4            MEMBER
    57         vidartf         4      COLLABORATOR
    10         betatim         4            MEMBER
    2     JohanMabille         4       CONTRIBUTOR
    58   vikasgarg1996         3       CONTRIBUTOR
    7         akhmerov         3            MEMBER
    50         parente         3            MEMBER
    24     fabihaahmed         3       CONTRIBUTOR
    33     jamesdbrock         3       CONTRIBUTOR
    40     kevin-bates         3            MEMBER
    36        jbweston         2            MEMBER
    32      jaipreet-s         2      COLLABORATOR
    34      jasongrout         2            MEMBER
    44          maresb         2              NONE
    38   joshuazeltser         2       CONTRIBUTOR
    41        krinsman         2       CONTRIBUTOR
    43        lresende         2            MEMBER
    52        t-makaro         2      COLLABORATOR
    1            Jamim         2       CONTRIBUTOR
    0         IMAM9AIS         2       CONTRIBUTOR
    5           afshin         2            MEMBER
    14        dalthviz         2       CONTRIBUTOR
    19         ednut15         2       CONTRIBUTOR
    23          esevan         1       CONTRIBUTOR
    51  

In [18]:
charts = []
for ii, (iorg, idata) in enumerate(closed_by_repo.replace(np.nan, 0).groupby(['org'])):
    title = f"Merges for {iorg} in the last {n_days} days"
    ch = alt.Chart(data=idata, width=1000, title=title).mark_bar().encode(
        x='username',
        y='closed',
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:                org        repo username  closed
    0  binder-examples  data-quilt  betatim       1,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:        org         repo  username  closed
    1  ipython    ipykernel   Carreau       1
    2  ipython    ipykernel     minrk       1
    3  ipython    ipykernel  willingc       1
    4  ipython  ipyparallel     minrk       2
    5  ipython      ipython   Carreau      10
    6  ipython    traitlets  rmorshea       2,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for ipython in the last 31 days',
    width: 1000
  }), Chart({
    data:         org                repo       username  closed
    7   jupyter              design        Carreau       1
    8   jupyter              design      blink1073       1
    9   jupyter       docker-stacks          minrk       6
    10  jupyter       docker-stacks        parente       5
    11  jupyter  enterprise_gateway    kevin-bates       8
    12  jupyter  enterprise_gateway       lresende       2
    13  jupyter          governance      choldgraf       1
    14  jupyter             jupyter      choldgraf       2
    15  jupyter             jupyter        parente       1
    16  jupyter        jupyter-book      choldgraf      13
    17  jupyter   jupyter-packaging      blink1073       2
    18  jupyter   jupyter-packaging        vidartf       1
    19  jupyter      jupyter-sphinx       akhmerov       5
    20  jupyter      jupyter-sphinx       jbweston       1
    21  jupyter   jupyter.github.io        Carreau       1
    22  jupyter   jupyter.github.io           Ruv7       1
    23  jupyter   jupyter.github.io         afshin       1
    24  jupyter   jupyter.github.io      choldgraf       2
    25  jupyter   jupyter.github.io      ellisonbg       1
    26  jupyter   jupyter.github.io     jasongrout       2
    27  jupyter      jupyter_client          MSeal       1
    28  jupyter      jupyter_client  SylvainCorlay       4
    29  jupyter      jupyter_client          minrk       3
    30  jupyter      jupyter_server  SylvainCorlay       3
    31  jupyter      jupyter_server       rolweber       1
    32  jupyter               nb2kg    kevin-bates       2
    33  jupyter           nbconvert          MSeal       8
    34  jupyter              nbdime        vidartf       1
    35  jupyter            nbviewer        parente       2
    36  jupyter            notebook  SylvainCorlay       2
    37  jupyter            notebook         ivanov       1
    38  jupyter            notebook       lresende       1
    39  jupyter            notebook      takluyver      10
    40  jupyter        ops-handbook         fperez       1
    41  jupyter           qtconsole     ccordoba12       4
    42  jupyter         repo2docker        betatim       7
    43  jupyter         repo2docker      choldgraf       1
    44  jupyter         repo2docker          minrk       3
    45  jupyter             roadmap      choldgraf       1
    46  jupyter           telemetry        Zsailer       6
    47  jupyter           telemetry     jaipreet-s       1
    48  jupyter           telemetry      yuvipanda       5,
    encoding: FacetedEncoding({
      x: X({
        field: 'username',
        type: 'nominal'
      }),
      y: Y({
        field: 'closed',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Merges for jupyter in the last 31 days',
    width: 1000
  }), Chart({
    data:                 org                repo         username  closed
    49  jupyter-widgets     

## Issues

Issues are **conversations** that happen on our GitHub repositories. Here's an
analysis of issues across the Jupyter organizations.

In [19]:
created = issues.query('state == "OPEN" and createdAt > @start_date and createdAt < @end_date')
closed = issues.query('state == "CLOSED" and closedAt > @start_date and closedAt < @end_date')

In [20]:
created_counts = created.groupby(['org', 'repo']).count()['number'].reset_index()
created_counts['org/repo'] = created_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = created_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=created_counts, title=f"Issues created in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                        repo  number  \
  0           ipython                   ipykernel       3   
  1           ipython                 ipyparallel       4   
  2           ipython                     ipython      10   
  3           ipython            ipython-in-depth       1   
  4           jupyter               docker-stacks       2   
  5           jupyter                     jupyter       3   
  6           jupyter                jupyter-book       1   
  7           jupyter              jupyter-sphinx       6   
  8           jupyter           jupyter.github.io       1   
  9           jupyter              jupyter_client       1   
  10          jupyter                jupyter_core       1   
  11          jupyter                       nb2kg       2   
  12          jupyter                   nbconvert       2   
  13          jupyter                    notebook      24   
  14          jupyter                   qtconsole       2   
  15          jupyter                 repo2docker       7   
  16          jupyter                   telemetry       1   
  17          jupyter                    testpath       1   
  18  jupyter-widgets                  ipyleaflet       3   
  19  jupyter-widgets                  ipywidgets      29   
  20  jupyter-widgets          jupyterlab-sidecar       4   
  21  jupyter-widgets                   pythreejs       1   
  22       jupyterhub                batchspawner       4   
  23       jupyterhub                      binder       3   
  24       jupyterhub                   binderhub      12   
  25       jupyterhub               dockerspawner       2   
  26       jupyterhub                  helm-chart       1   
  27       jupyterhub        jupyter-server-proxy       3   
  28       jupyterhub                  jupyterhub      20   
  29       jupyterhub  jupyterhub-deploy-teaching       1   
  30       jupyterhub                 kubespawner       4   
  31       jupyterhub           ldapauthenticator       4   
  32       jupyterhub         mybinder.org-deploy       3   
  33       jupyterhub         nativeauthenticator       1   
  34       jupyterhub              oauthenticator       2   
  35       jupyterhub               simplespawner       1   
  36       jupyterhub                 sudospawner       1   
  37       jupyterhub              systemdspawner       3   
  38       jupyterhub                team-compass       7   
  39       jupyterhub     the-littlest-jupyterhub       9   
  40       jupyterhub               traefik-proxy       1   
  41       jupyterhub                 wrapspawner       1   
  42       jupyterhub      zero-to-jupyterhub-k8s      11   
  43       jupyterlab           jupyter-renderers       2   
  44       jupyterlab                  jupyterlab      72   
  45       jupyterlab         jupyterlab-celltags       3   
  46       jupyterlab       jupyterlab-commenting       1   
  47       jupyterlab    jupyterlab-data-explorer      11   
  48       jupyterlab              jupyterlab-git       6   
  49       jupyterlab           jupyterlab-monaco       1   
  50       jupyterlab              jupyterlab-toc       1   
  51       jupyterlab             jupyterlab_apod       1   
  52       jupyterlab                team-compass       2   
  
                                   org/repo  
  0                       ipython/ipykernel  
  1                     ipython/ipyparallel  
  2                         ipython/ipython  
  3                ipython/ipython-in-depth  
  4                   jupyter/docker-stacks  
  5                         jupyter/jupyter  
  6                    jupyter/jupyter-book  
  7                  jupyter/jupyter-sphinx  
  8               jupyter/jupyter.github.io  
  9                  jupyter/jupyter_client  
  10                   jupyter/jupyter_core  
  11                          jupyter/nb2kg  
  12                      jupyter/nbconvert  
  13                       jupyter/notebook  

In [21]:
closed_counts = closed.groupby(['org', 'repo']).count()['number'].reset_index()
closed_counts['org/repo'] = closed_counts.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = closed_counts.sort_values(['org', 'number'], ascending=False)['repo'].values
alt.Chart(data=closed_counts, title=f"Issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y='number',
    color='org',
)

Chart({
  data:                 org                       repo  number  \
  0   binder-examples                 demo-julia       1   
  1           ipython                  ipykernel       1   
  2           ipython                    ipython      19   
  3           ipython                  traitlets       1   
  4           jupyter              docker-stacks       8   
  5           jupyter         enterprise_gateway       6   
  6           jupyter                    jupyter       2   
  7           jupyter               jupyter-book      16   
  8           jupyter             jupyter-sphinx      10   
  9           jupyter          jupyter.github.io       1   
  10          jupyter             jupyter_client       7   
  11          jupyter             jupyter_server       1   
  12          jupyter                      nb2kg       2   
  13          jupyter                  nbconvert       8   
  14          jupyter                     nbdime       3   
  15          jupyter                   nbviewer       3   
  16          jupyter                   notebook      15   
  17          jupyter                  qtconsole       2   
  18          jupyter                repo2docker       5   
  19          jupyter                  telemetry       3   
  20  jupyter-widgets                 ipyleaflet       4   
  21  jupyter-widgets                 ipywidgets      18   
  22  jupyter-widgets         jupyterlab-sidecar       1   
  23  jupyter-widgets               midicontrols       1   
  24  jupyter-widgets                  pythreejs       2   
  25  jupyter-widgets                   tutorial       1   
  26       jupyterhub               batchspawner       3   
  27       jupyterhub                  binderhub       4   
  28       jupyterhub              dockerspawner       3   
  29       jupyterhub       jupyter-server-proxy       1   
  30       jupyterhub                 jupyterhub       9   
  31       jupyterhub      kerberosauthenticator       1   
  32       jupyterhub                kubespawner       3   
  33       jupyterhub          ldapauthenticator       2   
  34       jupyterhub        mybinder.org-deploy       2   
  35       jupyterhub                nbgitpuller       1   
  36       jupyterhub               team-compass      16   
  37       jupyterhub    the-littlest-jupyterhub       5   
  38       jupyterhub              traefik-proxy       3   
  39       jupyterhub     zero-to-jupyterhub-k8s       9   
  40       jupyterlab  extension-cookiecutter-js       1   
  41       jupyterlab          jupyter-renderers       7   
  42       jupyterlab                 jupyterlab      59   
  43       jupyterlab      jupyterlab-commenting       1   
  44       jupyterlab   jupyterlab-data-explorer       7   
  45       jupyterlab             jupyterlab-git       5   
  46       jupyterlab             jupyterlab-toc       2   
  47       jupyterlab               team-compass       1   
  
                                  org/repo  
  0             binder-examples/demo-julia  
  1                      ipython/ipykernel  
  2                        ipython/ipython  
  3                      ipython/traitlets  
  4                  jupyter/docker-stacks  
  5             jupyter/enterprise_gateway  
  6                        jupyter/jupyter  
  7                   jupyter/jupyter-book  
  8                 jupyter/jupyter-sphinx  
  9              jupyter/jupyter.github.io  
  10                jupyter/jupyter_client  
  11                jupyter/jupyter_server  
  12                         jupyter/nb2kg  
  13                     jupyter/nbconvert  
  14                        jupyter/nbdime  
  15                      jupyter/nbviewer  
  16                      jupyter/notebook  
  17                     jupyter/qtconsole  
  18                   jupyter/repo2docker  
  19                     jupyter/telemetry  
  20            jupyter-widgets/ipyleaflet  
  21            jupyter-widgets/ipywidgets  
  22    

In [22]:
created_closed = pd.merge(created_counts.rename(columns={'number': 'created'}).drop(columns='org/repo'),
                          closed_counts.rename(columns={'number': 'closed'}).drop(columns='org/repo'),
                          on=['org', 'repo'], how='outer')

created_closed = pd.melt(created_closed, id_vars=['org', 'repo'], var_name="kind", value_name="count").replace(np.nan, 0)

In [23]:
charts = []
for org in github_orgs:
    # Pick the top 10 repositories
    this_issues = created_closed.query('org == @org')
    top_repos = this_issues.groupby(['repo']).sum().sort_values(by='count', ascending=False).head(10).index
    ch = alt.Chart(this_issues.query('repo in @top_repos'), width=120).mark_bar().encode(
        x=alt.X("kind", axis=alt.Axis(labelFontSize=15, title="")), 
        y=alt.Y('count', axis=alt.Axis(titleFontSize=15, labelFontSize=12)),
        color='kind',
        column=alt.Column("repo", header=alt.Header(title=f"Issue activity, last {n_days} days for {org}", titleFontSize=15, labelFontSize=12))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:             org                       repo     kind  count
    43   jupyterlab          jupyter-renderers  created    2.0
    44   jupyterlab                 jupyterlab  created   72.0
    45   jupyterlab        jupyterlab-celltags  created    3.0
    46   jupyterlab      jupyterlab-commenting  created    1.0
    47   jupyterlab   jupyterlab-data-explorer  created   11.0
    48   jupyterlab             jupyterlab-git  created    6.0
    49   jupyterlab          jupyterlab-monaco  created    1.0
    50   jupyterlab             jupyterlab-toc  created    1.0
    52   jupyterlab               team-compass  created    2.0
    63   jupyterlab  extension-cookiecutter-js  created    0.0
    107  jupyterlab          jupyter-renderers   closed    7.0
    108  jupyterlab                 jupyterlab   closed   59.0
    109  jupyterlab        jupyterlab-celltags   closed    0.0
    110  jupyterlab      jupyterlab-commenting   closed    1.0
    111  jupyterlab   jupyterlab-data-explorer   closed    7.0
    112  jupyterlab             jupyterlab-git   closed    5.0
    113  jupyterlab          jupyterlab-monaco   closed    0.0
    114  jupyterlab             jupyterlab-toc   closed    2.0
    116  jupyterlab               team-compass   closed    1.0
    127  jupyterlab  extension-cookiecutter-js   closed    1.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for jupyterlab',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:          org              repo     kind  count
    0    ipython         ipykernel  created    3.0
    1    ipython       ipyparallel  created    4.0
    2    ipython           ipython  created   10.0
    3    ipython  ipython-in-depth  created    1.0
    54   ipython         traitlets  created    0.0
    64   ipython         ipykernel   closed    1.0
    65   ipython       ipyparallel   closed    0.0
    66   ipython           ipython   closed   19.0
    67   ipython  ipython-in-depth   closed    0.0
    118  ipython         traitlets   closed    1.0,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      column: Column({
        field: 'repo',
        header: Header({
          labelFontSize: 12,
          title: 'Issue activity, last 31 days for ipython',
          titleFontSize: 15
        }),
        type: 'nominal'
      }),
      x: X({
        axis: Axis({
          labelFontSize: 15,
          title: ''
        }),
        field: 'kind',
        type: 'nominal'
      }),
      y: Y({
        axis: Axis({
          labelFontSize: 12,
          titleFontSize: 15
        }),
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    width: 120
  }), Chart({
    data:                  org                repo     kind  count
    18   jupyter-widgets          ipyleaflet  created    3.0
    19   jupyter-widgets          ipywidgets  created   29.0
    20   jupyter-widgets  jupyterlab-sidecar  created    4.0
    21   jupyter-widgets           pythreejs  created    1.0
    59   jupyter-widgets        midicontrols  created    0.0
    60   jupyter-widgets            tutorial  created    0.0
    82   jupyter-widgets          ipyleaflet   closed    4.0
    83   jupyter-widgets          ipywidgets   closed   18.0
    84   jupyter-widgets  jupyterlab-sidecar   closed    1.0
    85   jupyter

In [24]:
# Set to datetime
for kind in ['createdAt', 'closedAt']:
    closed.loc[:, kind] = pd.to_datetime(closed[kind])
    
closed.loc[:, 'time_open'] = closed['closedAt'] - closed['createdAt']
closed.loc[:, 'time_open'] = closed['time_open'].dt.total_seconds()

In [25]:
time_open = closed.groupby(['org', 'repo']).agg({'time_open': 'median'}).reset_index()
time_open['time_open'] = time_open['time_open'] / (60 * 60 * 24)
time_open['org/repo'] = time_open.apply(lambda a: a['org'] + '/' + a['repo'], axis=1)
sorted_vals = time_open.sort_values(['org', 'time_open'], ascending=False)['repo'].values
alt.Chart(data=time_open, title=f"Time to close for issues closed in the last {n_days} days").mark_bar().encode(
    x=alt.X('repo', sort=alt.Sort(sorted_vals.tolist())),
    y=alt.Y('time_open', title="Median Days Open"),
    color='org',
)

Chart({
  data:                 org                       repo   time_open  \
  0   binder-examples                 demo-julia    6.085058   
  1           ipython                  ipykernel   34.939491   
  2           ipython                    ipython    6.007373   
  3           ipython                  traitlets    0.001817   
  4           jupyter              docker-stacks   29.893924   
  5           jupyter         enterprise_gateway    6.011175   
  6           jupyter                    jupyter  115.932251   
  7           jupyter               jupyter-book   81.046586   
  8           jupyter             jupyter-sphinx    6.007975   
  9           jupyter          jupyter.github.io    2.754988   
  10          jupyter             jupyter_client    9.848565   
  11          jupyter             jupyter_server    6.048576   
  12          jupyter                      nb2kg   10.164954   
  13          jupyter                  nbconvert  323.023571   
  14          jupyter                     nbdime   10.806308   
  15          jupyter                   nbviewer   27.010463   
  16          jupyter                   notebook   16.943368   
  17          jupyter                  qtconsole    6.688837   
  18          jupyter                repo2docker    2.884236   
  19          jupyter                  telemetry    0.930949   
  20  jupyter-widgets                 ipyleaflet    8.565475   
  21  jupyter-widgets                 ipywidgets    7.011470   
  22  jupyter-widgets         jupyterlab-sidecar   16.910301   
  23  jupyter-widgets               midicontrols    3.001574   
  24  jupyter-widgets                  pythreejs    2.115446   
  25  jupyter-widgets                   tutorial    2.426273   
  26       jupyterhub               batchspawner  247.168843   
  27       jupyterhub                  binderhub    1.544005   
  28       jupyterhub              dockerspawner    3.970845   
  29       jupyterhub       jupyter-server-proxy    0.577951   
  30       jupyterhub                 jupyterhub   19.687465   
  31       jupyterhub      kerberosauthenticator   35.502523   
  32       jupyterhub                kubespawner    5.357789   
  33       jupyterhub          ldapauthenticator   13.380573   
  34       jupyterhub        mybinder.org-deploy   12.693681   
  35       jupyterhub                nbgitpuller    0.008333   
  36       jupyterhub               team-compass   72.433993   
  37       jupyterhub    the-littlest-jupyterhub   48.672465   
  38       jupyterhub              traefik-proxy    0.844363   
  39       jupyterhub     zero-to-jupyterhub-k8s    8.797431   
  40       jupyterlab  extension-cookiecutter-js    0.977616   
  41       jupyterlab          jupyter-renderers  328.480972   
  42       jupyterlab                 jupyterlab    4.221053   
  43       jupyterlab      jupyterlab-commenting   14.237836   
  44       jupyterlab   jupyterlab-data-explorer    5.240394   
  45       jupyterlab             jupyterlab-git   32.688519   
  46       jupyterlab             jupyterlab-toc    1.095272   
  47       jupyterlab               team-compass    7.733113   
  
                                  org/repo  
  0             binder-examples/demo-julia  
  1                      ipython/ipykernel  
  2                        ipython/ipython  
  3                      ipython/traitlets  
  4                  jupyter/docker-stacks  
  5             jupyter/enterprise_gateway  
  6                        jupyter/jupyter  
  7                   jupyter/jupyter-book  
  8                 jupyter/jupyter-sphinx  
  9              jupyter/jupyter.github.io  
  10                jupyter/jupyter_client  
  11                jupyter/jupyter_server  
  12                         jupyter/nb2kg  
  13                     jupyter/nbconvert  
  14                        jupyter/nbdime  
  15                      jupyter/nbviewer  
  16                      jupyter/notebook  
  17                     jupyter/qtco

### A list of recent issues

Below is a list of issues with recent activity in each repository. If they seem of interest
to you, click on their links and jump in to participate!

In [26]:
# Add comment count data to issues and PRs
comment_counts = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'issue_id'])
    .count().iloc[:, 0].to_frame()
)
comment_counts.columns = ['n_comments']
comment_counts = comment_counts.reset_index()

In [27]:
n_plot = 5
tabs = widgets.Tab(children=[])
for ii, (org, idata) in enumerate(comment_counts.groupby('org')):
    issue_md = []
    issue_md.append(f"#### {org}")
    issue_md.append("")
    for repo, i_issues in idata.groupby('repo'):
        issue_md.append(f"##### [{org}/{repo}](https://github.com/{org}/{repo})")

        top_issues = i_issues.sort_values('n_comments', ascending=False).head(n_plot)
        top_issue_list = pd.merge(issues, top_issues, left_on=['org', 'repo', 'number'], right_on=['org', 'repo', 'issue_id'])
        for _, issue in top_issue_list.sort_values('n_comments', ascending=False).head(n_plot).iterrows():
            user_name = issue['author']
            user_url = author_url(user_name)
            issue_number = issue['number']
            issue_html = issue['url']
            issue_title = issue['title']

            text = f"* [(#{issue_number})]({issue_html}): _{issue_title}_ by **[@{user_name}]({user_url})**"
            issue_md.append(text)

    issue_md.append('')
    md_html = HTML(markdown('\n'.join(issue_md)))
    
    children = list(tabs.children)
    children.append(HTML(markdown('\n'.join(issue_md))))
    tabs.children = tuple(children)
    tabs.set_title(ii, org)
    
display(Markdown(f"Here are the top {n_plot} active issues in each repository in the last {n_days} days"))
display(tabs)

Here are the top 5 active issues in each repository in the last 31 days

## Commenters across repositories

These are commenters across all issues and pull requests in the last several days.
These are colored by the commenter's association with the organization. For information
about what these associations mean, [see this StackOverflow post](https://stackoverflow.com/a/28866914/1927102).

In [28]:
commentors = (
    comments
    .query("createdAt > @start_date and createdAt < @end_date")
    .groupby(['org', 'repo', 'author', 'authorAssociation'])
    .count().rename(columns={'id': 'count'})['count']
    .reset_index()
    .sort_values(['org', 'count'], ascending=False)
)

In [29]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(commentors.groupby(['org'])):
    title = f"Top {n_plot} commentors for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'count': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('count', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='count',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:          author  count authorAssociation
    1     choldgraf      3            MEMBER
    2   cornhundred      2              NONE
    0       betatim      1            MEMBER
    3  davidanthoff      1       CONTRIBUTOR
    4   matheusmota      1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'count',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 commentors for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:                     author  count authorAssociation
    4                  Carreau     66            MEMBER
    10                   MSeal      7       CONTRIBUTOR
    76               tacaswell      6       CONTRIBUTOR
    69                 pdeitel      5              NONE
    70                rmorshea      5      COLLABORATOR
    64                   minrk      4            MEMBER
    26               blink1073      4            MEMBER
    55             kevin-bates      3       CONTRIBUTOR
    15        RafalSkolasinski      3              NONE
    24                 anntzer      3       CONTRIBUTOR
    9           LucianaMarques      3       CONTRIBUTOR
    42                gpotter2      3       CONTRIBUTOR
    38                fehiepsi      2              NONE
    40            flying-sheep      2       CONTRIBUTOR
    37                ericdill      2              NONE
    68                oscargus      2              NONE
    30              ccordoba12      2       CONTRIBUTOR
    79              thrasibule      2              NONE
    46              jamesmyatt      2              NONE
    8                    Jamim      2       CONTRIBUTOR
    31                cottrell      2       CONTRIBUTOR
    82                voronind      2              NONE
    83                 wenjuno      2              NONE
    84                willingc      2            MEMBER
    51                jseabold      2       CONTRIBUTOR
    60                m0nzderr      1              NONE
    57                   kribu      1              NONE
    59                 leogout      1              NONE
    58               leifwalsh      1              NONE
    62              matthewgdv      1              NONE
    56                khughitt      1              NONE
    54                   kernc      1              NONE
    53               keflavich      1              NONE
    61                  mag-al      1              NONE
    0   AbsurdMagpieScrutinies      1              NONE
    63         maxwellmckinnon      1              NONE
    65                 mjkvaak      1              NONE
    66                    mx73      1              NONE
    67                 nschloe      1              NONE
    71                 scopatz      1       CONTRIBUTOR
    72               smerzbach      1              NONE
    73            sowmyakavali      1              NONE
    74          stuartthomas25      1              NONE
    75                 sunpoet      1       CONTRIBUTOR
    77                 tbeadle      1              NONE
    78              texadactyl      1              NONE
    80                  torbjo      1              NONE
    81             vinklibrary      1              NONE
    85               yamasampo      1              NONE
    52              juanis2112      1       CONTRIBUTOR,
    encoding: FacetedEncoding({
      color

## First responders

First responders are the first people to respond to a new issue in one of the repositories.
The following plots show first responders for recently-created issues.

In [30]:
first_comments = []
for (org, repo, issue_id), i_comments in comments.groupby(['org', 'repo', 'issue_id']):
    ix_min = pd.to_datetime(i_comments['createdAt']).idxmin()
    first_comment = i_comments.loc[ix_min]
    if isinstance(first_comment, pd.DataFrame):
        first_comment = first_comment.iloc[0]
    first_comments.append(first_comment)
first_comments = pd.concat(first_comments, axis=1).T

In [31]:
first_responder_counts = first_comments.groupby(['org', 'author', 'authorAssociation'], as_index=False).\
    count().rename(columns={'id': 'n_first_responses'}).sort_values(['org', 'n_first_responses'], ascending=False)


In [32]:
n_plot = 50
charts = []
for ii, (iorg, idata) in enumerate(first_responder_counts.groupby(['org'])):
    title = f"Top {n_plot} first responders for {iorg} in the last {n_days} days"
    idata = idata.groupby('author', as_index=False).agg({'n_first_responses': 'sum', 'authorAssociation': 'first'})
    idata = idata.sort_values('n_first_responses', ascending=False).head(n_plot)
    ch = alt.Chart(data=idata.head(n_plot), width=1000, title=title).mark_bar().encode(
        x='author',
        y='n_first_responses',
        color=alt.Color('authorAssociation', scale=alt.Scale(domain=author_types, range=author_colors))
    )
    charts.append(ch)
alt.hconcat(*charts)

HConcatChart({
  hconcat: [Chart({
    data:         author  n_first_responses authorAssociation
    0      betatim                  1            MEMBER
    1    choldgraf                  1            MEMBER
    2  cornhundred                  1              NONE,
    encoding: FacetedEncoding({
      color: Color({
        field: 'authorAssociation',
        scale: Scale({
          domain: ['MEMBER', 'CONTRIBUTOR', 'COLLABORATOR', 'NONE'],
          range: ['rgb(211.82745098039214, 211.82745098039214, 211.82745098039214, 256.0)', 'rgb(166.98562091503268, 229.89803921568628, 166.98562091503268, 256.0)', 'rgb(96.37647058823529, 192.75294117647059, 96.37647058823529, 256.0)', 'rgb(0.0, 100.3921568627451, 0.0, 256.0)']
        }),
        type: 'nominal'
      }),
      x: X({
        field: 'author',
        type: 'nominal'
      }),
      y: Y({
        field: 'n_first_responses',
        type: 'quantitative'
      })
    }),
    mark: 'bar',
    title: 'Top 50 first responders for binder-examples in the last 31 days',
    width: 1000
  }), Chart({
    data:             author  n_first_responses authorAssociation
    2          Carreau                 30            MEMBER
    54        willingc                  2            MEMBER
    4            Jamim                  2       CONTRIBUTOR
    0     Benjamin-Lee                  1              NONE
    42      matthewgdv                  1              NONE
    31          jdatin                  1              NONE
    32  joaomarcosgris                  1              NONE
    33       keflavich                  1              NONE
    34           kernc                  1              NONE
    35     kevin-bates                  1       CONTRIBUTOR
    36        khughitt                  1              NONE
    37           kribu                  1              NONE
    38       leifwalsh                  1              NONE
    39         leogout                  1              NONE
    40        m0nzderr                  1              NONE
    41          mag-al                  1              NONE
    44         mjkvaak                  1              NONE
    43           minrk                  1            MEMBER
    29        janusson                  1              NONE
    45            mx73                  1              NONE
    46        oscargus                  1              NONE
    47    sowmyakavali                  1              NONE
    48  stuartthomas25                  1              NONE
    49       tacaswell                  1              NONE
    50         tbeadle                  1              NONE
    51          torbjo                  1              NONE
    52     vinklibrary                  1              NONE
    53         wenjuno                  1              NONE
    30      jasongrout                  1            MEMBER
    28      jamesmyatt                  1              NONE
    1       BryceHaley                  1              NONE
    27        jaamarks                  1              NONE
    3           Evidlo                  1              NONE
    5   LucianaMarques                  1       CONTRIBUTOR
    6            MSeal                  1       CONTRIBUTOR
    7       OriolAbril                  1       CONTRIBUTOR
    8    PietroPasotti                  1              NONE
    9         Scowley4                  1              NONE
    10       Shaliniyv                  1              NONE
    11            Suor                  1              NONE
    12      alexmojaki                  1              NONE
    13      anentropic                  1              NONE
    14       blink1073                  1            MEMBER
    15       brakedust                  1              NONE
    16     carrutstick                  1              NONE
    17      ccordoba12                  1       CONTRIBUTOR
    18     craymichael                  1              NONE
    19    davidmakovoz                  1       

In [33]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>